[View in Colaboratory](https://colab.research.google.com/github/JohnnyUrosevic/Machine-Learning-Shenanigans/blob/master/Iris.ipynb)

In [1]:
import tensorflow as tf
import numpy as np

Collecting data for features and labels

In [2]:
from google.colab import files
CSV_COLUMNS = ["sepal-length", "sepal-width", "petal-length", "petal-width", "class"]
CSV_DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [""]]

In [3]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, record_defaults=CSV_DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop('class')
    return features, label

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(filenames=data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=10000)

  dataset = dataset.map(parse_csv)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, label = iterator.get_next()
  return features, label

Build Model

In [4]:
sepal_length = tf.feature_column.numeric_column('sepal-length')
sepal_width = tf.feature_column.numeric_column('sepal-width')
petal_length = tf.feature_column.numeric_column('petal-length')
petal_width = tf.feature_column.numeric_column('petal-width')

model = tf.estimator.LinearClassifier(feature_columns=[sepal_length, sepal_width, petal_length, petal_width],
                                     n_classes=3, label_vocabulary=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk2jeabzn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1dc44a4f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Get uploaded training and test data

Run training

In [5]:
files.upload()

!ls
!pwd

adult.data.csv	adult.test.csv	datalab
/content


In [6]:
model.train(input_fn=lambda: input_fn("train.csv", 1000, True, 32))

Parsing train.csv
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpk2jeabzn/model.ckpt.


NotFoundError: ignored

Evaluate

In [42]:
results = model.evaluate(input_fn=lambda: input_fn("test.csv", 1, False, 32))

for key in sorted(results):
  print('%s: %s' % (key, results[key]))

Parsing test.csv
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-28-21:17:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmtp0ja2m/model.ckpt-3688
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-28-21:17:09
INFO:tensorflow:Saving dict for global step 3688: accuracy = 1.0, average_loss = 0.066359416, global_step = 3688, loss = 2.1235013
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3688: /tmp/tmpmtp0ja2m/model.ckpt-3688
accuracy: 1.0
average_loss: 0.066359416
global_step: 3688
loss: 2.1235013


Make Predictions

In [43]:
predictions = model.predict(input_fn=lambda: input_fn("test.csv", 1, False, 32))

for pred_dict in predictions:
    print("Prediction:%s Probability:%.1f%%" % (pred_dict['classes'][0], 100 * pred_dict['probabilities'][pred_dict['class_ids'][0]]))

Parsing test.csv
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmtp0ja2m/model.ckpt-3688
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction:b'Iris-virginica' Probability:96.9%
Prediction:b'Iris-virginica' Probability:81.1%
Prediction:b'Iris-virginica' Probability:71.3%
Prediction:b'Iris-setosa' Probability:99.8%
Prediction:b'Iris-virginica' Probability:98.7%
Prediction:b'Iris-setosa' Probability:98.9%
Prediction:b'Iris-versicolor' Probability:84.2%
Prediction:b'Iris-setosa' Probability:98.6%
Prediction:b'Iris-setosa' Probability:94.1%
Prediction:b'Iris-versicolor' Probability:98.0%
Prediction:b'Iris-setosa' Probability:99.3%
Prediction:b'Iris-virginica' Probability:89.0%
Prediction:b'Iris-virginica' Probability:91.8%
Prediction:b'Iris-virginica' Probability:99.9%
Prediction:b'Iris-setosa' Probability:100.0%
Prediction:b'Iris-virgini